In [ ]:
!az login

# bonus: rdp by winRM https://docs.microsoft.com/en-us/azure/virtual-machines/windows/winrm

In [67]:
# images
windows = 'Win2019Datacenter'
# windows = 'Win2019Server'
ubuntu = 'UbuntuLTS'

class VM:
    def __init__(self, name, group, location, kwargs):
        self.name = name
        self.group = group
        self.location = location
        self.kwargs = kwargs
    
    def argstring(self) -> str:
        return f'-n {self.name} -g {self.group} -l {self.location} ' + ' '.join([f'--{arg} {val}' for arg, val in self.kwargs.items()])

def create_group(name: str, location: str) -> None:
    !az group create -n $name -l $location

def create_vnet(name: str, group: str, location: str, addr_prf: str, subnet: str) -> None:
    !az network vnet create -n $name -g $group -l $location --address-prefix $addr_prf --subnet-name $subnet

def create_vm(vm: VM) -> None:
    text = vm.argstring()
    !az vm create $text

def create_all(location: str, group: str, vnet: str, addr_prf: str, subnet: str, vms: list[dict[str, str]]) -> None:

    # Resource Group
    create_group(group, location)

    # Virtual Network + Subnet
    create_vnet(vnet, group, location, addr_prf, subnet)

    # Virtual Machines
    for vm in vms:
        create_vm(vm)

def create_vnet_peering(name: str, group: str, src: str, dst_name: str) -> None:
    # get dst vnet id
    dst = !az network vnet show -g $group -n $src --query id --out tsv
    dst = dst[0]

    !az network vnet peering create -n $name -g $group --vnet-name $src --remove-vnet $dst --allow-vnet-access

In [ ]:
# Europe
eulocation = 'northeurope'
eugroup = 'eugroup'
euvnet = 'euvnet'
euaddr_prf = '10.1.0.0/16'
eusubnet = 'eusubnet'

euvms = [
    VM('VM_Windows', eugroup, eulocation, {'image': windows, 'admin-user': 'azureuser', 'admin-password': 'linux4everlinux4ever!', 'public-ip-sku': 'Standard'}),
    VM('VM_Ubuntu', eugroup, eulocation, {'image': ubuntu, 'admin-user': 'azureuser', 'ssh-key-values': 'azure.pub', 'public-ip-sku': 'Standard'})
]

In [ ]:
# Canada
calocation = 'canadaeast'
cagroup = 'cagroup'
cavnet = 'cavnet'
caaddr_prf = '10.2.0.0/16'
casubnet = 'casubnet'

cavms = [
    VM('VM_Ubuntu', cagroup, calocation, {'image': ubuntu, 'admin-user': 'azureuser', 'ssh-key-values': 'azure.pub', 'public-ip-sku': 'Standard'})
]

In [ ]:
create_group(eugroup, eulocation)
create_group(cagroup, calocation)

create_vnet(euvnet, eugroup, eulocation, euaddr_prf, eusubnet)
create_vnet(cavnet, cagroup, calocation, caaddr_prf, casubnet)

linkvnet_eu_ca = 'linkvnet_eu_ca'
linkvnet_ca_eu = 'linkvnet_ca_eu'

!az network vnet peering create -n $linkvnet_eu_ca -g $eugroup --vnet-name $euvnet --remote-vnet $ca_net_id --allow-vnet-access
!az network vnet peering create -n $linkvnet_ca_eu -g $cagroup --vnet-name $cavnet --remote-vnet $eu_net_id --allow-vnet-access

create_vm(euvms[0]) # Windows Datacenter 2019
create_vm(euvms[1]) # UbuntuVM
create_vm(cavms[0]) # UbuntuVM

# create_all(eulocation, eugroup, euvnet, euaddr_prf, eusubnet, euvms) 
# create_all(calocation, cagroup, cavnet, caaddr_prf, casubnet, cavms)

In [ ]:
# CLUTTER

# AADDS
# _name = 'zwedditDS'
# _group = 'eugroup'
# _domain = 'zweddit.onmicrosoft.com'
# _replica = ''

# !az ad ds create -n $_name -g $_group --domain $_domain --replica-sets $_replica --tls-v1 'Disabled'

# az network p2s-vpn-gateway create

# 2 rules for ICMP
# 1 "gateway subnet"
# 1 virtual network gateway + 1 public ip name
# certificates >.>

In [ ]:
# Destroy
!az group delete -n $eugroup -y
!az group delete -n $cagroup -y